In [ ]:
import models
from data_generator import DataGenerator

from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

In [ ]:
input_shape = [64, 64, 3]

# model = models.create_first_model(input_shape)
# model = models.create_pretrained_mobileNet(input_shape, True)
model = models.create_second_model(input_shape)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

data_generator = DataGenerator("/home/olga/my/work/aid/data/images/training/image_2/",
                                       "/home/olga/my/work/aid/data/labels/training/label_2/")
data_generator.read_data()
data_generator.shuffle()

In [ ]:
batch_size = 12
train_generator = train_datagen.flow(
    x=data_generator.Xtrain,y = data_generator.Ytrain,
    batch_size=batch_size)

validation_generator = test_datagen.flow(x=data_generator.Xtest, y=data_generator.Ytest,
    batch_size=batch_size)

In [ ]:
nb_train_samples = 5000
nb_validation_samples = 800
epochs = 100

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('second_model_100epoch.h5')

Draw confusion matrix

In [ ]:
# model.load_weights('first_model_500epoch.h5')

In [ ]:
y_pred = []
for answer in model.predict(validation_generator.x):
    y_pred.append(np.argmax(answer))

In [ ]:
len(y_pred), len(data_generator.Ytest)

In [ ]:
confusion_matrix(data_generator.Ytest, y_pred)

visualise errors


In [ ]:
number_images = 0
for i, y_pred_i in enumerate(y_pred):
    if y_pred_i == data_generator.Ytest[i]:
        continue
    number_images += 1
    if number_images >= 10:
        break
    print("real class - {}, predicted class - {}".format(data_generator.number2object[data_generator.Ytest[i]],
                                                        data_generator.number2object[y_pred_i]))
    plt.imshow(data_generator.Xtest[i])
    plt.show()
    

how you can see, several errors link with not good detections (espesially if you see one more time objects with widht less than 5)